In [1]:
import numpy as np
import pandas as pd
from astropy.table import Table
from astropy.io.fits import getdata
from astropy.time import Time
from astropy.io import fits

In [2]:
df = pd.read_csv('csv/full_merged.csv')
df.head()

,SL Target,SEQID,Module,Primary Target,Exposure (s),RA,DEC,PA,Unnamed: 8,In old sheet?,Notes
0,1RXSJ180408.9-342058,90102003002,A,SAX_J1808d4m3658,49267.8,272.1324,-36.9541,159.4761,1,Y,"Strong SL, unknown?"
1,1RXSJ180408.9-342058,90102003004,A,SAX_J1808d4m3658,21508.8,272.1424,-36.9510,168.1145,1,Y,"Strong, unident SL"
2,1RXSJ180408.9-342058,90102003004,B,SAX_J1808d4m3658,21508.8,272.1424,-36.9510,168.1145,1,Y,"Strong, unident SL"
3,2MASX J09172716-6456271,60401020002,A,IRAS_09149m6206,110899.0,138.9791,-62.3074,50.7186,1,N,SL on both
4,2MASX J09172716-6456271,60401020002,B,IRAS_09149m6206,110899.0,138.9791,-62.3074,50.7186,0,N,SL in both


In [3]:
greenlist = ['SL Target', 'SEQID', 'Module', 'Primary Target', 'Exposure (s)', 'RA',
       'DEC', 'Notes']
for col in df.columns:
    if col not in greenlist:
        df = df.drop(axis=1, labels=col)
        
# Drop everything with NaN in the SL Target column
df = df.dropna(subset=['SL Target'])
df.head()

,SL Target,SEQID,Module,Primary Target,Exposure (s),RA,DEC,Notes
0,1RXSJ180408.9-342058,90102003002,A,SAX_J1808d4m3658,49267.8,272.1324,-36.9541,"Strong SL, unknown?"
1,1RXSJ180408.9-342058,90102003004,A,SAX_J1808d4m3658,21508.8,272.1424,-36.9510,"Strong, unident SL"
2,1RXSJ180408.9-342058,90102003004,B,SAX_J1808d4m3658,21508.8,272.1424,-36.9510,"Strong, unident SL"
3,2MASX J09172716-6456271,60401020002,A,IRAS_09149m6206,110899.0,138.9791,-62.3074,SL on both
4,2MASX J09172716-6456271,60401020002,B,IRAS_09149m6206,110899.0,138.9791,-62.3074,SL in both


In [4]:

# Set the "public" cutoff date to be June 1, 2020
cutoff_t = Time('2020-06-01T00:00:00.123456789', format='isot', scale='utc')
cutoff_mjd = cutoff_t.mjd

numaster = Table.read('fits/numaster.fits', format = 'fits')
    
dn = numaster.to_pandas()

greenlist = ['NAME', 'RA', 'DEC', 'TIME', 'END_TIME',
       'OBSID', 'EXPOSURE_A', 'EXPOSURE_B', 'PUBLIC_DATE']
for col in dn.columns:
    if col not in greenlist:
        dn.drop(col, axis=1, inplace=True)

dn['OBSID'] = pd.to_numeric(dn['OBSID'])
dn.columns

dn['PUBLIC'] = np.where((dn['PUBLIC_DATE']<cutoff_mjd)&(dn['PUBLIC_DATE'] > 0), 'yes', 'no')
dn

,NAME,RA,DEC,TIME,END_TIME,OBSID,EXPOSURE_A,EXPOSURE_B,PUBLIC_DATE,PUBLIC
0,b'3C273_pcs11 ...,187.2386,2.0479,56109.875778,56109.941750,15011001,2881.8646,2882.2901,56533,yes
1,b'3C273_pcs12 ...,187.2038,2.0472,56109.941750,56110.011194,15012001,2880.8013,2894.2422,56533,yes
2,b'3C273_pcs13 ...,187.2367,2.0157,56110.011194,56110.080639,15013001,2572.5306,2575.3012,56533,yes
3,b'3C273_pcs14 ...,187.3142,2.0520,56110.080639,56110.146611,15014001,2607.3166,2609.0228,56533,yes
4,b'3C273_pcs15 ...,187.2341,2.1272,56110.146611,56110.212583,15015001,2844.8164,2847.9784,56533,yes
...,...,...,...,...,...,...,...,...,...,...
7357,b'J0612m06 ...,93.0010,-6.3691,-99.000000,-99.000000,60601033,-70000.0000,-70000.0000,-99,no
7358,b'ESO495mG005 ...,124.8114,-25.1879,-99.000000,-99.000000,60668002,-100000.0000,-100000.0000,-99,no
7359,b'ESO420mG013 ...,63.4570,-32.0069,-99.000000,-99.000000,60668003,-100000.0000,-100000.0000,-99,no
7360,b'MRK_1218 ...,129.5456,24.8953,-99.000000,-99.000000,60601034,-75000.0000,-75000.0000,-99,no


In [5]:

df['TIME'] = [-1. for x in range(len(df['SEQID']))]
df['END_TIME'] = [-1. for x in range(len(df['SEQID']))]

for si in np.unique(df['SEQID']):
#    print(dn.loc[dn['OBSID'] == si, 'TIME'].values[0])
    df.loc[df['SEQID'] == si, 'TIME'] = dn.loc[dn['OBSID'] == si, 'TIME'].values[0]
    df.loc[df['SEQID'] == si, 'END_TIME'] = dn.loc[dn['OBSID'] == si, 'END_TIME'].values[0]
df.head()

,SL Target,SEQID,Module,Primary Target,Exposure (s),RA,DEC,Notes,TIME,END_TIME
0,1RXSJ180408.9-342058,90102003002,A,SAX_J1808d4m3658,49267.8,272.1324,-36.9541,"Strong SL, unknown?",57127.042444,57128.309805
1,1RXSJ180408.9-342058,90102003004,A,SAX_J1808d4m3658,21508.8,272.1424,-36.9510,"Strong, unident SL",57151.969528,57152.427861
2,1RXSJ180408.9-342058,90102003004,B,SAX_J1808d4m3658,21508.8,272.1424,-36.9510,"Strong, unident SL",57151.969528,57152.427861
3,2MASX J09172716-6456271,60401020002,A,IRAS_09149m6206,110899.0,138.9791,-62.3074,SL on both,58323.306356,58325.788995
4,2MASX J09172716-6456271,60401020002,B,IRAS_09149m6206,110899.0,138.9791,-62.3074,SL in both,58323.306356,58325.788995


In [9]:
obj_type = pd.read_csv('csv/target_info.csv')
green_list = ['SL Target', 'Type']
for col in obj_type.columns:
    if col not in green_list:
        if col in obj_type.columns:
            obj_type = obj_type.drop(axis=1,labels=col)
obj_type.head()

,SL Target,Type
0,1RXSJ180408.9-342058,LMXB-NS
1,2MASX J09172716-6456271,AGN
2,3C 273,AGN
3,4U 0614+091,LMXB-NS
4,4U 1323-619,LMXB-NS


In [56]:
# Add target classification and target type

df2 = df.copy()
nrows = len(df2['SEQID'])
print(nrows)

classification= ['SL' for x in range(nrows) ]
targ_type= ['??' for x in range(nrows) ]


df2.insert(loc=0, column='Classification', value=classification)
df2.insert(loc=2, column='Target Type', value=targ_type)

for target, ttype in zip(obj_type['SL Target'], obj_type['Type']):
    df2.loc[df2['SL Target'] == target, 'Target Type'] = ttype

    
for id in ['Unkn', 'Complex', 'Faint', 'GR']:
    df2.loc[df2['SL Target'] == id, 'Classification'] = id
    df2.loc[df2['SL Target'] == id, 'SL Target'] = '??'



df2

727


,Classification,SL Target,Target Type,SEQID,Module,Multi,Primary Target,Exposure (s),RA,DEC,Notes,TIME,END_TIME
0,SL,1RXSJ180408.9-342058,LMXB-NS,90102003002,A,N,SAX_J1808d4m3658,49267.8,272.1324,-36.9541,"Strong SL, unknown?",57127.042444,57128.309805
1,SL,1RXSJ180408.9-342058,LMXB-NS,90102003004,A,N,SAX_J1808d4m3658,21508.8,272.1424,-36.9510,"Strong, unident SL",57151.969528,57152.427861
2,SL,1RXSJ180408.9-342058,LMXB-NS,90102003004,B,N,SAX_J1808d4m3658,21508.8,272.1424,-36.9510,"Strong, unident SL",57151.969528,57152.427861
3,SL,2MASX J09172716-6456271,AGN,60401020002,A,N,IRAS_09149m6206,110899.0,138.9791,-62.3074,SL on both,58323.306356,58325.788995
4,SL,2MASX J09172716-6456271,AGN,60401020002,B,N,IRAS_09149m6206,110899.0,138.9791,-62.3074,SL in both,58323.306356,58325.788995
...,...,...,...,...,...,...,...,...,...,...,...,...,...
722,SL,X1908+075,HMXB-NS,90001010002,B,N,XTE_J1859p083,19838.6,284.7785,8.2742,Faint SL,57111.719528,57112.181333
723,SL,XTE J1855-026,HMXB-NS,30360001002,A,N,Swift_J1843d5m0343,75612.3,280.9001,-3.6847,"Weak SL, second source present",57887.861912,57889.552884
724,SL,XTE J1855-026,HMXB-NS,30360001002,B,N,Swift_J1843d5m0343,75612.3,280.9001,-3.6847,Weak edge SL in both,57887.861912,57889.552884
725,SL,XTE J1855-026,HMXB-NS,40301004002,B,N,KES_75,95271.8,281.5856,-3.0002,Weak SL on both?,58013.788995,58016.202190


In [60]:
count = df2['SEQID'].value_counts()

multi = ['N' for x in range(nrows) ]


for seq in count.keys():
    if count[seq] > 1:
        print(seq, count[seq])
        this_seq =df2[df2['SEQID'] == seq]
        mods = this_seq['Module'].value_counts()
        for mod in mods.keys():
            if mods[mod] > 1:
                df2.loc[(df2['SEQID'] == seq) & (df2['Module'] == mod), 'Multi'] = 'Y'
    
    


30102054006 7
40201001002 5
40501004004 4
90201020002 4
90001005002 4
80202013005 4
60061279002 4
30301024002 4
30001033002 4
30001130002 4
30601007002 3
40111002002 3
90501343002 3
80002040002 3
40001014022 3
40001014023 3
90301322001 3
40001014013 3
40101015002 3
30001012002 3
90201027002 3
40014007001 3
40101012002 3
30002003002 3
40101012004 3
80101050002 3
30102038002 3
30102038007 3
30102038006 3
30102038009 3
80001046002 3
40001015005 3
40001015003 3
40001014012 3
90401331002 3
90601310002 3
60301010002 3
30401018002 3
80301311002 3
30102054002 3
80402320002 3
80202013003 3
40112003002 3
30160001002 3
30102054004 3
30160002002 3
80001041002 3
90102003006 3
30401019002 3
80002014002 3
30002021011 2
10502010003 2
90001002002 2
90302321002 2
30001044002 2
30101026002 2
10502010001 2
30002021009 2
30460005002 2
40301004002 2
30401017002 2
90401314002 2
10610025001 2
10402002004 2
30460020002 2
80302312001 2
90201021010 2
40101010002 2
30002021002 2
40101012003 2
40001015007 2
803023

In [64]:
df2[df2['Multi']=='Y']

,Classification,SL Target,Target Type,SEQID,Module,Multi,Primary Target,Exposure (s),RA,DEC,Notes,TIME,END_TIME
17,SL,4U 1624-490,Eclipsing pulsar,30001012002,B,Y,NuSTARJ163433m473841,16599.9,248.6712,-47.6363,GR + multiple SL. Not sure about the 4U 1624-4...,56374.354944,56374.764666
20,SL,4U 1624-490,Eclipsing pulsar,30001033002,A,Y,CXOUJ163750d8m465545,51986.4,249.4897,-46.9014,"Two SL, one unident",57050.219528,57051.282028
21,SL,4U 1624-490,Eclipsing pulsar,30001033002,B,Y,CXOUJ163750d8m465545,51986.4,249.4897,-46.9014,Multiple overlapping,57050.219528,57051.282028
30,SL,4U 1624-490,Eclipsing pulsar,30160001002,B,Y,CXOUJ163709d7m473157,50437.2,249.3137,-47.5722,GX 340 + unknown on B,57184.615361,57185.747305
31,SL,4U 1624-490,Eclipsing pulsar,30160002002,B,Y,CXOUJ163543d2m473411,99512.6,248.9436,-47.5917,GX 340 + unknown on B,57180.990361,57183.330639
...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,Unkn,??,??,80402320002,B,Y,SGR_1806m20,39917.1,272.1367,-20.4465,"Multiple, whole FoV, unident",58409.806356,58410.841079
714,Unkn,??,??,90001005002,B,Y,IGRJ17544m2619,79252.2,268.6311,-26.3045,Multiple SL sources. Fainter ones unknown,57101.035500,57102.834111
715,Unkn,??,??,90001005002,A,Y,IGRJ17544m2619,79252.2,268.6311,-26.3045,Multiple SL sources. Fainter ones unknown,57101.035500,57102.834111
719,SL,X1908+075,HMXB-NS,30401018002,B,Y,4U_1907p09,78859.8,287.3659,9.8237,Strong SL (weak in A),58331.528579,58333.320245


In [45]:
mods

A    4
B    3
Name: Module, dtype: int64

In [15]:
tab = Table.from_pandas(df2)

tab.write('test.fits')

In [99]:
print(df2.columns)
arr = df2.to_numpy()

Index(['Classification', 'SL Target', 'Target Type', 'SEQID', 'Module',
       'Primary Target', 'Exposure (s)', 'RA', 'DEC', 'Notes', 'TIME',
       'END_TIME'],
      dtype='object')


In [107]:

# Make the columns the silly way here:

classification = fits.Column(name='CLASSIFICATION', format='10A', array=arr[:, 0])
target = fits.Column(name='SL_SOURCE', format='10A', array=arr[:,1])
ttype = fits.Column(name='TYPE', format='4A', array=arr[:,2])
obsid = fits.Column(name='OBSID', format='12A', array=arr[:,3])
mod = fits.Column(name='MODULE', format='2A', array=arr[:,4])
primary = fits.Column(name='PRIMARY_TARGET', format = '30A', array=arr[:,5])
exposure = fits.Column(name='EXPOSURE', format = 'E', array=[float(e) for e in arr[:,6]])
ra = fits.Column(name='RA', format='E', array =  arr[:, 7])
dec = fits.Column(name='DEC', format='E', array =  arr[:, 8])
tstart = fits.Column(name='TIME', format='E', array = arr[:, 10])
tend = fits.Column(name='TIME_END', format='E', array = arr[:, 11])
notes = fits.Column(name='NOTES', format='30A', array = arr[:, 9])



In [115]:
hdu = fits.BinTableHDU.from_columns([classification, target, obsid, mod, primary, exposure, ra, dec, tstart, tend, notes])

hdu.writeto('fits/straycats.fits', overwrite=True)

In [117]:
straycat = getdata('fits/straycats.fits')
straycat.names

['CLASSIFICATION',
 'SL_SOURCE',
 'OBSID',
 'MODULE',
 'PRIMARY_TARGET',
 'EXPOSURE',
 'RA',
 'DEC',
 'TIME',
 'TIME_END',
 'NOTES']